In [1]:
!pip install sqlalchemy

In [2]:
import pandas as pd
import sqlalchemy as sa
import sys

## Подключение к БД

In [3]:
sys.path.append(r'C:\Users\Sviridov\Desktop\Тестовые\Pet')
from task_db import db_config

In [4]:
connection_string1 = 'postgresql://{user}:{pwd}@{host}:{port}/sql_class'.format(**db_config)
connection_string2 = 'postgresql://{user}:{pwd}@{host}:{port}/exercises'.format(**db_config)

engine1 = sa.create_engine(connection_string1, connect_args={'sslmode':'disable'})
engine2 = sa.create_engine(connection_string2, connect_args={'sslmode':'disable'})

In [5]:
def get_sql_data(query:str, engine) -> pd.DataFrame:
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Запросы

### 1. Вывести всех членов семьи у которых ИМЯ (Не фамилия+имя, а именно ИМЯ) длинее чем 6 символов.

In [6]:
task1 = '''
select left(member_name, (position(' ' in member_name) - 1)) as name
from "family".familymembers
where position(' ' in member_name) >= 8
'''

In [7]:
get_sql_data(task1, engine1).head()

,name
0,Headley
1,Wednesday
2,Constance


### 2. Покажите комнаты (таблицы rooms) в которых есть ТВ или есть кухня с интернетом. Результат отсортируйте в порядке убывания цены.

In [8]:
task2 = '''
select * 
from airbnb.rooms
where (home_type = 'Private room' and has_tv = true and has_internet = false and has_kitchen = false) 
or (home_type = 'Private room' and has_tv = false and has_internet = true and has_kitchen = true)
order by price desc
'''

In [9]:
get_sql_data(task2, engine1).head(10)

,id,home_type,address,has_tv,has_internet,has_kitchen,has_air_con,price,owner_id,latitude,longitude
0,3,Private room,"10027, West 128th Street, New York",True,False,False,True,150,2,40.8090,-73.9419
1,44,Private room,"10024, West 87th Street, New York",False,True,True,False,135,21,40.7901,-73.9793
2,22,Private room,"11217, Saint Marks Avenue, New York",True,False,False,True,130,20,40.6807,-73.9771
3,47,Private room,"11101, 21st Street, New York",True,False,False,True,130,26,40.7477,-73.9474
4,36,Private room,"11215, 10th Street, New York",True,False,False,True,89,32,40.6687,-73.9878
5,35,Private room,"11216, Jefferson Avenue, New York",True,False,False,False,70,32,40.6831,-73.9547
6,29,Private room,"10034, Post Avenue, New York",True,False,False,False,44,27,40.8648,-73.9211


### 3. Какие комнаты бронировали (reservations) пользователи у которых не верифицирован email?

In [10]:
task3 = '''
select r.room_id 
from airbnb.reservations r
join airbnb.users u on r.user_id = u.id 
where u.email_verified_at is null
'''

In [11]:
get_sql_data(task3, engine1).head()

,room_id
0,1
1,8
2,49


### 4. Выведите id и признак наличия интернета в помещении. Если интернет в сдаваемом жилье присутствует, то выведите «YES», иначе «NO».

In [12]:
task4 = '''
select id,
       case
       	   when has_internet = true then 'YES'
       	   else 'NO'
       end as internet    
from airbnb.rooms
'''

In [13]:
get_sql_data(task4, engine1).head()

,id,internet
0,1,YES
1,2,YES
2,3,NO
3,5,YES
4,6,YES


### 5. Вывести названия классов, в которых учатся больше 5 мальчиков. (hard)

In [14]:
task5 = '''
select c.name 
from schedule."class" c
join schedule.student_in_class sc on c.id = sc.class
join schedule.student s on sc.student = s.id
where s.last_name like '%v'
group by c.name 
having count(s.last_name) > 5
'''

In [15]:
get_sql_data(task5, engine1).head()

,name
0,11 A
1,10 B
2,10 A


### 6. Выведите идентификаторы преподавателей, которые хотя бы один раз за всё время преподавали в каждом из одиннадцатых классов.(hard)

In [16]:
task6 = '''
select s.teacher
from schedule.schedule s
join schedule."class" c on s.class = c.id 
where c.name = '11 B' or c.name = '11 A'
group by s.teacher
having count(distinct (c.name)) > 1
'''

In [17]:
get_sql_data(task6, engine1).head()

,teacher
0,2
1,3
2,6
3,8
4,10


### 7. Вывести названия классов, в которых учатся больше 5 мальчиков.

In [18]:
task7 = '''
select c.name 
from schedule."class" c
join schedule.student_in_class sc on c.id = sc.class
join schedule.student s on sc.student = s.id
where s.last_name like '%v'
group by c.name 
having count(s.last_name) > 5
'''

In [19]:
get_sql_data(task7, engine1).head()

,name
0,11 A
1,10 B
2,10 A


### 8. Выведите идентификаторы преподавателей, которые хотя бы один раз за всё время преподавали в каждом из одиннадцатых классов.

In [20]:
task8 = '''
select s.teacher
from schedule.schedule s
join schedule."class" c on s.class = c.id 
where c.name = '11 B' or c.name = '11 A'
group by s.teacher
having count(distinct (c.name)) > 1
'''

In [21]:
get_sql_data(task8, engine1).head()

,teacher
0,2
1,3
2,6
3,8
4,10


### 9. Выведите имя самого старшего человека. Если таких несколько, то выведите их всех.

In [22]:
task9_1 = '''
select member_name, 
       birthday,  
       AGE(birthday)
from "family".familymembers 
where AGE(birthday) in (
                        select MAX(AGE(birthday))
                        from "family".familymembers
                        )
                        '''

In [23]:
get_sql_data(task9_1, engine1).head()

,member_name,birthday,age
0,Headley Quincey,1960-05-13,23886 days


In [24]:
task9_2 = '''
select member_name, 
       birthday,
       extract(year from current_date) - extract(year from birthday) as age
from "family".familymembers 
where extract(year from current_date) - extract(year from birthday) in (
                                                                        select MAX(extract(year from current_date) - extract(year from birthday))
                                                                        from "family".familymembers
                                                                        )
                                                                        '''

In [25]:
get_sql_data(task9_2, engine1).head()

,member_name,birthday,age
0,Headley Quincey,1960-05-13,65.0


### 10. Выведите идентификаторы преподавателей, которые хотя бы один раз за всё время преподавали в каждом из одиннадцатых классов.

In [26]:
task10 = '''
with A11 as (
             select distinct(teacher) as teacher_id
             from schedule.schedule s
             join schedule."class" c
             on s."class" = c.id     
             where c.name = '11 A'    
            ),
     B11 as (
             select distinct(teacher) as teacher_id
             from schedule.schedule s
             join schedule."class" c
             on s."class" = c.id     
             where c.name = '11 B'    
            )
select A11.teacher_id
from A11 
join B11 on A11.teacher_id = B11.teacher_id
'''

In [27]:
get_sql_data(task10, engine1).head()

,teacher_id
0,2
1,3
2,6
3,8
4,10


### 11. Для каждой комнаты, которую снимали как минимум 1 раз, найдите имя человека, снимавшего ее последний раз, и дату, когда он выехал.

In [28]:
task11 = '''
with combo as (
               select room_id,
                      max(end_date) as last_end_date
               from airbnb.reservations
               group by room_id
               having count(start_date) > 1 
              )
select r.room_id, 
       u."name",
       r.end_date
from airbnb.reservations r
join airbnb.users u on r.user_id=u.id
join combo on r.room_id = combo.room_id and r.end_date = combo.last_end_date
'''

In [29]:
get_sql_data(task11, engine1).head()

,room_id,name,end_date
0,1,Michael Douglas,2019-02-04 12:00:00
1,21,Alan Rickman,2020-02-29 10:00:00
2,19,Samuel L. Jackson,2020-05-02 10:00:00
3,5,Russell Crowe,2020-05-15 10:00:00
4,49,Bruce Willis,2020-06-11 10:00:00


### 12. Определить группы товаров, которые не приобретались в 2005 году

In [30]:
task12 = '''
select good_type_name
from "family".goodtypes
where good_type_id in(
                      select "type"
                      from "family".goods
                      where good_id not in(
                                           select distinct(good) as good
                                           from "family".payments
                                           where extract(year from date) = 2005
                                          ) 
                     )
                     '''

In [31]:
get_sql_data(task12, engine1).head()

,good_type_name
0,communal payments
1,food
2,delicacies
3,equipment


### 13. Напишите запрос, который вычисляет разницу между зарплатой каждого сотрудника и средней зарплатой в его департаменте.

In [32]:
task13 = '''
with sal_avgsal as (
                    select "name",
                           salary,
                           round(avg(salary) over (partition by department_id), 0) as avg_salary
                    from sber.employee
                   )
select "name",
       avg_salary - salary as difference
from sal_avgsal
'''

In [33]:
get_sql_data(task13, engine2).head()

,name,difference
0,Crystal Herrera,43118.0
1,Richard Ward,-32156.0
2,Christopher Lee,25953.0
3,Sheila Reese,-67008.0
4,Gilbert Hardy,42553.0


### 8. test

In [34]:
task14 = '''select date_trunc('month', date)::date,
           count(product_key),
           LAG(COUNT(product_key)) OVER (ORDER BY date_trunc('month', date)),
           count(product_key) + LAG(COUNT(product_key)) OVER (ORDER BY date_trunc('month', date))
           from alphabank.purchase
           group by date_trunc('month', date)
           order by date_trunc('month', date)'''

In [35]:
get_sql_data(task14, engine2).head(10)

,date_trunc,count,lag,?column?
0,2022-06-01,1,NaN,NaN
1,2022-08-01,1,1.0,2.0
2,2023-03-01,6,1.0,7.0
3,2023-04-01,2,6.0,8.0
4,2023-05-01,2,2.0,4.0
5,2023-06-01,3,2.0,5.0
6,2023-10-01,2,3.0,5.0
7,2023-12-01,1,2.0,3.0
8,2024-01-01,7,1.0,8.0
9,2024-02-01,8,7.0,15.0
